In [265]:
from random import random
import numpy as np
from itertools import combinations
import math
import random
import time as tm

In [266]:
# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

In [267]:
# decrement the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g


In [268]:
### create a graph with planted clique of size = clique_size

# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# clique_size: clique size

def generate_graph(num_vertices, pr, clique_size):
    start_generate = tm.time()
    def plant_clique():
        for i in range(clique_size):
            for j in range(i + 1, clique_size):
                for l in range(j + 1, clique_size):
                    a = clique_vertices[i]
                    # print(a)
                    vec[a] += 1
                    b = clique_vertices[j]
                    # print(b)
                    vec[b] += 1
                    c = clique_vertices[l]
                    # print(c)
                    vec[c] += 1
                    add_all(a, b, c, g)

    g = np.array([np.array([np.array([0 for i in range(0, num_vertices)]) for i in range(num_vertices)]) for i in
                  range(num_vertices)])
    vec = np.array([0 for i in range(0, num_vertices)])

    # Set edges
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                a = np.random.uniform(0, 1, 1)
                # every edge is included independently with probability 1/2
                if a < pr:
                    vec[i] += 1
                    vec[j] += 1
                    vec[k] += 1
                    add_all(i, j, k, g)

    # clique_vertices = random.sample(range(num_vertices), clique_size) # is is wrong??; would violate the clique size constraint
    
    clique_vertices = np.random.choice(range(num_vertices), clique_size, replace=False)

    plant_clique()

    return g, vec, clique_vertices


In [269]:
# calculate the total number of edges on a graph with num_vertices vertices
def find_num_edges(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [270]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    # active_count: number of vertices that are associated with at least 1 edge
    active_count = np.count_nonzero(vec)
    # number of edges
    edge_sum = find_num_edges(g, num_vertices)
    if edge_sum == math.comb(active_count, 3):
        return True
    return False

In [271]:
# remove edge
def remove_edges(g, num_vertices, vec, curr):
    vec[curr] = 0
    graph_copy = g.copy()
    for j in range(0, num_vertices):
        for k in range(0, num_vertices):
            if graph_copy[curr, j, k] == 1:
                vec[j] -= 1
                vec[k] -= 1
                remove_all(curr, j, k, graph_copy)
    return graph_copy, vec

### Generate a graph

In [272]:
# driver program

n = 100 # number of vertices
p = 0.5 # probablity of edge being included
k = 10 # clique size

# generate graph
start_generate = tm.time()
res = generate_graph(n, p, k)

# G graph with planted clique
# V: vector storing the number of edges associated with each vertice in the graph
# clique_vertices: the set of clique vertices
G, V, clique_vertices = res[0],  res[1], res[-1]

# print(f"graph: {G}\n")
G_0 = G.copy()
print(f"edge-occurence vector: {V}\n")
print(f"set of clique vertices after removal phase: {clique_vertices}\n")

time_generate = np.round(tm.time()-start_generate,3)
# print("Time taken to generate graph: ", "seconds")


edge-occurence vector: [2526 2460 2478 2495 2419 2378 2436 2385 2394 2441 2385 2417 2459 2428
 2396 2467 2444 2449 2397 2403 2437 2418 2423 2409 2441 2445 2390 2386
 2402 2389 2381 2456 2407 2510 2509 2425 2519 2455 2502 2403 2448 2513
 2448 2396 2394 2462 2449 2422 2466 2449 2388 2351 2448 2419 2425 2428
 2454 2373 2400 2391 2463 2451 2432 2472 2416 2459 2417 2373 2417 2428
 2422 2434 2386 2434 2370 2415 2382 2492 2390 2484 2398 2450 2475 2355
 2480 2431 2535 2444 2462 2383 2499 2466 2419 2413 2450 2417 2389 2431
 2423 2462]

set of clique vertices after removal phase: [16 73 25 74 23  0  1  2 84 91]



### Removal Phase

In [273]:
start_removal = tm.time()
iter = 0
removed = []

clique = is_clique(G, V, n)
while not clique:
    
    iter += 1

    curr = -1
    idx_sorted = np.argsort(V)
    
    print(f"i = {iter}\nindex sorted: {idx_sorted}")
    print(f"number of edges associated with each vertex: {V}")
    # print(f'current set of clique vertices: {clique_vertices}')

    for idx in range(n):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            removed.append(curr)
            break
    print(f"vertex removed: {curr} number of edges: {V[curr]}\n")
    A = remove_edges(G, n, V, curr)
    # print(A[1])
    G = A[0]

    clique = is_clique(G, V, n)
print("clique!!!")
time_remove = np.round(tm.time()-start_removal,3)

i = 1
index sorted: [51 83 74 57 67  5 30 76 89  7 10 72 27 50 96 29 78 26 59 44  8 43 14 18
 80 58 28 39 19 32 23 93 75 64 95 66 68 11 21 92 53  4 70 47 98 22 54 35
 13 69 55 85 97 62 73 71  6 20  9 24 87 16 25 40 42 52 17 49 46 81 94 61
 56 37 31 65 12  1 88 99 45 60 48 91 15 63 82  2 84 79 77  3 90 38 34 33
 41 36  0 86]
number of edges associated with each vertex: [2526 2460 2478 2495 2419 2378 2436 2385 2394 2441 2385 2417 2459 2428
 2396 2467 2444 2449 2397 2403 2437 2418 2423 2409 2441 2445 2390 2386
 2402 2389 2381 2456 2407 2510 2509 2425 2519 2455 2502 2403 2448 2513
 2448 2396 2394 2462 2449 2422 2466 2449 2388 2351 2448 2419 2425 2428
 2454 2373 2400 2391 2463 2451 2432 2472 2416 2459 2417 2373 2417 2428
 2422 2434 2386 2434 2370 2415 2382 2492 2390 2484 2398 2450 2475 2355
 2480 2431 2535 2444 2462 2383 2499 2466 2419 2413 2450 2417 2389 2431
 2423 2462]
vertex removed: 51 number of edges: 2351

i = 2
index sorted: [51 83  5 67 57 30 74 89 27 43 10 18 78 96 72 76 59 26 28 

In [274]:
iter

91

In [275]:
sorted(removed)

[3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 24,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

### Inclusion Phase

In [276]:
full_vertices = np.arange(n)
full_vertices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [277]:
included = np.setdiff1d(full_vertices, removed)
included

array([ 0,  1,  2, 16, 23, 25, 73, 84, 91])

In [278]:
def inclusion_phase(removed, in_set, G_0):
    
    def connected(idx, in_set):
        for j in range(len(in_set)):
                for k in range(j+1, len(in_set)):
                    if G_0[idx, in_set[j], in_set[k]] != 1:
                        print(f" vertex {idx} is not connected to all clique vertices")  
                        return False
        return True
                        
    for idx in removed:
        if connected(idx, in_set):
            print(f"add {idx} to clique!")
            in_set = np.append(in_set, idx)


    return in_set
    

In [279]:
start_include = tm.time()
res = inclusion_phase(removed, included, G_0)
time_include = np.round(tm.time()-start_include,3)

 vertex 51 is not connected to all clique vertices
 vertex 83 is not connected to all clique vertices
 vertex 5 is not connected to all clique vertices
 vertex 30 is not connected to all clique vertices
 vertex 67 is not connected to all clique vertices
 vertex 7 is not connected to all clique vertices
 vertex 76 is not connected to all clique vertices
add 74 to clique!
 vertex 57 is not connected to all clique vertices
 vertex 27 is not connected to all clique vertices
 vertex 10 is not connected to all clique vertices
 vertex 39 is not connected to all clique vertices
 vertex 78 is not connected to all clique vertices
 vertex 96 is not connected to all clique vertices
 vertex 59 is not connected to all clique vertices
 vertex 72 is not connected to all clique vertices
 vertex 89 is not connected to all clique vertices
 vertex 14 is not connected to all clique vertices
 vertex 18 is not connected to all clique vertices
 vertex 70 is not connected to all clique vertices
 vertex 98 is n

In [280]:
print(f"Set of clique vertices after generation phase: {clique_vertices}\n")
print(f"Set of clique vertices after removal phase: {included}, number of elements = {len(included)}\n")
print(f"set of vertices removed: {removed}, number of elements = {len(removed)}\n")
print(f"Set of clique vertices after inclusion phase: {res}")

Set of clique vertices after generation phase: [16 73 25 74 23  0  1  2 84 91]

Clique vertices after removal phase: [ 0  1  2 16 23 25 73 84 91], number of elements = 9

set of vertices removed: [51, 83, 5, 30, 67, 7, 76, 74, 57, 27, 10, 39, 78, 96, 59, 72, 89, 14, 18, 70, 98, 64, 92, 97, 11, 75, 4, 87, 15, 50, 42, 85, 43, 26, 31, 80, 65, 58, 94, 66, 24, 60, 71, 55, 19, 47, 52, 32, 77, 53, 49, 93, 79, 54, 81, 3, 45, 22, 12, 33, 46, 21, 8, 28, 29, 90, 63, 41, 82, 56, 13, 61, 62, 68, 37, 48, 36, 95, 38, 35, 17, 20, 86, 88, 99, 9, 44, 69, 40, 34, 6], number of elements = 91

Clique vertices after inclusion phase: [ 0  1  2 16 23 25 73 84 91 74]
